In [1]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
import numpy as np
import pandas as pd

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_181"; Java(TM) SE Runtime Environment (build 1.8.0_181-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.181-b13, mixed mode)
  Starting server from /home/ricardo/.local/lib/python3.5/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpsaji9ik5
  JVM stdout: /tmp/tmpsaji9ik5/h2o_ricardo_started_from_python.out
  JVM stderr: /tmp/tmpsaji9ik5/h2o_ricardo_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,America/Mexico_City
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.2
H2O cluster version age:,26 days
H2O cluster name:,H2O_from_python_ricardo_atrc1f
H2O cluster total nodes:,1
H2O cluster free memory:,1.707 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [3]:
spotify = h2o.import_file("s3://proyectomineria/data/consolidated_train_table/part-00000-acf23e82-8c41-458f-9399-57e2a260de4b-c000.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
spotify["is_churn"] = spotify["is_churn"].asfactor()
spotify["gender"] = spotify["gender"].asfactor()

In [5]:
spotify

msno,is_churn,city,bd,gender,registered_via,registered_init_time,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
++4RuqBw0Ss6bQU4oMxaRlbBPoWzoEiIZaxPM04Y4+U=,0,1,0,,7,2014-07-14T00:00:00.000Z,1.48398e+09,5.73333,0.933333,0.733333,0.666667,6.46667,13.0667,1978.66,41,30,149,149,1,1.48162e+09,nan,0
+/namlXq+u3izRjHCFJV4MgqcXcLidZYszVsROOq/y4=,0,15,31,male,9,2006-06-03T00:00:00.000Z,1.48755e+09,29.1333,1.26667,1.4,1.26667,33.6,61.1333,9395.27,34,30,149,149,1,1.48311e+09,nan,0
+0/X9tkmyHyet9X80G6GTrDFHnJqvai8d1ZPhayT0os=,0,9,31,male,9,2004-03-30T00:00:00.000Z,1.48764e+09,12.4667,1.86667,1.06667,3.26667,67.8,19.8667,17219,34,30,149,149,1,1.48311e+09,nan,0
+09YGn842g6h2EZUXe0VWeC4bBoCbDGfUboitc0vIHw=,0,15,29,male,9,2008-03-22T00:00:00.000Z,1.48751e+09,2.33333,0.6,0.333333,0.666667,33,27.8,8571.42,34,30,149,149,1,1.48311e+09,nan,0
+0RJtbyhoPAHPa+34MkYcE2Ox0cjMgJOTXMXVBYgkJE=,1,13,29,female,3,2012-06-12T00:00:00.000Z,1.48615e+09,2.4,0.866667,0.866667,1.33333,18.2,20.5333,4813.96,32,410,1788,1788,0,1.45273e+09,nan,0
+0jTOa6KGPk1vtNTwRDMZc/McUo41AeuwV3ndo54Y+Q=,0,5,24,female,9,2014-03-20T00:00:00.000Z,1.48739e+09,10.7333,4.4,0.933333,0.533333,19.6,30.9333,5353.78,23,30,149,149,1,1.4805e+09,nan,0
+0l+FDuhyjaZnu0APnrg5L9QqgaRw4RmdQMvqOtKDmU=,0,13,32,male,3,2015-03-16T00:00:00.000Z,1.48743e+09,12.3333,1.6,1.66667,0.933333,40.6,46.6667,10597.8,37,30,149,149,1,1.48257e+09,nan,0
+0l/WkoOIugT69NYawwewSLZjIJ17kHIpDdWqcp53RI=,0,5,0,,3,2013-02-27T00:00:00.000Z,1.48656e+09,1,0.4,0.333333,0.133333,21.5333,20.6667,5413.59,40,30,149,149,1,1.48302e+09,nan,0.2
+2Df04hr61UUJijMM2xR97gtoQWWDJpnJVKQ7VMYN9o=,0,6,31,female,9,2008-04-17T00:00:00.000Z,1.48764e+09,1.26667,0.466667,0.333333,1.86667,281.667,15,48663,36.4,30,167.6,167.6,1,1.48131e+09,nan,0.2
+2eLsQv6T46iKwO+m+r6OFI2X3Oc9dGBMdti2COAe4w=,0,1,0,,7,2012-12-17T00:00:00.000Z,1.48719e+09,8.93333,2.73333,1.33333,3,11.4,23.2667,3921.46,41,30,99,99,1,1.47938e+09,nan,0


In [6]:
predictors = ["city", "gender", "registered_via", "registered_init_time", "date", "num_25", "num_50", "num_75", "num_985", "num_100", "num_unq", "total_secs","payment_method_id", "payment_plan_days", "plan_list_price", "actual_amount_paid", "is_auto_renew", "transaction_date", "is_cancel"] 
response = "is_churn"

In [7]:
train, valid = spotify.split_frame(ratios = [0.8], seed=1234)

In [8]:
bin_num = [8,16,32,64,128,256,512,1024,2048,4096]
label = ["8","16","32","64","128","256","512","1024","2048","4096"]

### Gradient Boosting Machine

Gradient Boosting Machine (GBM) is a forward learning ensemble method. The guiding heuristic is that good predictive results can be obtained through increasingly refined approximations. H2O's GBM sequentially builds regression trees on all the features of the data set ina fully distributed way- each tree built in parallel. In some way, this model learns from the errors of the previous trees.

In [9]:
for key, num in enumerate(bin_num):
    spotify_gbm = H2OGradientBoostingEstimator(nbins_cats = num, seed=1234)
    spotify_gbm.train(x=predictors, y=response, training_frame=train, validation_frame=valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [10]:
print(label[key], 'training score', spotify_gbm.auc(train=True))
print(label[key], 'validation score', spotify_gbm.auc(valid=True))

4096 training score 0.976811171802014
4096 validation score 0.9713729284966914


In [11]:
final_gbm_predictions = spotify_gbm.predict(valid[1:])

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [12]:
final_gbm_predictions[:]

predict,p0,p1
0,0.996169,0.00383067
0,0.995891,0.00410929
0,0.995197,0.00480288
0,0.966933,0.0330673
1,0.201176,0.798824
0,0.997387,0.00261345
0,0.862844,0.137156
0,0.769958,0.230042
0,0.996278,0.00372208
0,0.997261,0.00273875


Now we are going to modify the model to try to use only data from the user logs. Transactions data has a very different time frame. And data from members is static.

In [13]:
spotify1 = h2o.import_file("s3://proyectomineria/data/consolidated_train_table/part-00000-acf23e82-8c41-458f-9399-57e2a260de4b-c000.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [14]:
spotify1["is_churn"] = spotify1["is_churn"].asfactor()

In [15]:
predictors1 = ["date", "num_25", "num_50", "num_75", "num_985", "num_100", "num_unq", "total_secs"] 
response1 = "is_churn"

In [16]:
train1, valid1 = spotify1.split_frame(ratios = [0.8], seed=1234)

In [17]:
bin_num1 = [8,16,32,64,128,256,512,1024,2048,4096]
label1 = ["8","16","32","64","128","256","512","1024","2048","4096"]

In [18]:
for key, num in enumerate(bin_num1):
    spotify_gbm1 = H2OGradientBoostingEstimator(nbins_cats = num, seed=1234)
    spotify_gbm1.train(x=predictors1, y=response1, training_frame=train1, validation_frame=valid1)

gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [19]:
print(label[key], 'training score', spotify_gbm1.auc(train=True))
print(label[key], 'validation score', spotify_gbm1.auc(valid=True))

4096 training score 0.698655270278644
4096 validation score 0.6912409028948672


Although this model has a greater error it is also more useful because tracks the user recent behavior.

### Distributed Random Forest

Distributed Random Forest (DRF) is a powerful classification and regression tool. When given a set of data, DRF generates a forest of classification or regression trees, rather than a single classification or regression tree. Each of these trees is a weak learner built on a subset of rows and columns. More trees will reduce the variance. Both classification and regression take the average prediction over all of their trees to make a final prediction, whether predicting for a class or numeric value.

In [20]:
rf_v1 = H2ORandomForestEstimator(
    model_id="rf_covType_v1",
    ntrees=200,
    stopping_rounds=2,
    score_each_iteration=True,
seed=1000000)

In [21]:
rf_v1.train(x=predictors, y=response, training_frame=train, validation_frame=valid)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [22]:
rf_v1.score_history()

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2018-12-18 12:03:50,0.016 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2018-12-18 12:03:50,0.981 sec,1.0,0.227669,1.523522,0.798407,0.169438,9.211052,0.055617,0.227365,1.517085,0.797626,0.168610,9.094388,0.054459
2,,2018-12-18 12:03:51,1.551 sec,2.0,0.218741,1.253010,0.787966,0.200050,9.385622,0.054728,0.196285,0.583053,0.866890,0.373463,11.433029,0.057349
3,,2018-12-18 12:03:52,2.057 sec,3.0,0.214341,1.119995,0.814802,0.235330,9.591126,0.057697,0.186320,0.361566,0.913646,0.481753,12.235982,0.045662
4,,2018-12-18 12:03:52,2.526 sec,4.0,0.209565,0.942319,0.835633,0.289562,9.827775,0.055687,0.181875,0.242142,0.933719,0.568774,13.084951,0.045853
5,,2018-12-18 12:03:53,3.022 sec,5.0,0.207588,0.745359,0.844578,0.346388,9.913552,0.056094,0.180970,0.169229,0.940811,0.633732,12.873991,0.048042
6,,2018-12-18 12:03:53,3.594 sec,6.0,0.203492,0.644184,0.860318,0.380434,10.260552,0.055432,0.178770,0.147094,0.949699,0.660755,12.778007,0.048467
7,,2018-12-18 12:03:54,4.249 sec,7.0,0.199184,0.551303,0.873043,0.415153,10.621417,0.058472,0.176712,0.133014,0.954687,0.681003,13.096247,0.044324
8,,2018-12-18 12:03:55,5.305 sec,8.0,0.196447,0.495156,0.884656,0.440040,10.765676,0.053510,0.175579,0.128331,0.956034,0.695068,13.631497,0.045981
9,,2018-12-18 12:03:56,6.092 sec,9.0,0.194783,0.430545,0.892602,0.465450,10.927244,0.052962,0.175817,0.124522,0.957061,0.693029,13.238735,0.043070


In [23]:
rf_v2 = H2ORandomForestEstimator(
    model_id="rf_covType_v2",
    ntrees=200,
    max_depth=30,
    stopping_rounds=2,
    stopping_tolerance=0.01,
    score_each_iteration=True,
seed=1234)

In [24]:
rf_v2.train(x=predictors, y=response, training_frame=train, validation_frame=valid)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [25]:
final_rf_predictions = rf_v2.predict(valid[1:])

drf prediction progress: |████████████████████████████████████████████████| 100%


In [26]:
final_rf_predictions

predict,p0,p1
0,1,0
0,0.996679,0.003321
0,0.99721,0.00279006
0,0.844581,0.155419
1,0.137406,0.862594
0,0.996768,0.00323234
0,0.958092,0.0419077
0,0.730893,0.269107
0,0.996242,0.00375827
0,0.997174,0.00282552


In [27]:
print(label[key], 'training score', rf_v2.auc(train=True))
print(label[key], 'validation score', rf_v2.auc(valid=True))

4096 training score 0.9403659723572353
4096 validation score 0.963956318230576


In [28]:
print(label[key], 'training score', rf_v1.auc(train=True))
print(label[key], 'validation score', rf_v1.auc(valid=True))

4096 training score 0.9599919542557266
4096 validation score 0.9692609712431121


The best model so far is the GBM, we will use these to predict the test file

In [55]:
spotify_test = h2o.import_file("s3://proyectomineria/data/resumen_final_test/part-00000-326c4568-e87c-4af5-9c77-6ee2aa5d17ae-c000.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [56]:
spotify_test

avg_num_unq,date,bd,payment_plan_days,city,avg_num_50,registered_init_time,msno,avg_num_75,plan_list_price,actual_amount_paid,avg_num_25,avg_num_100,membership_expire_date,is_churn,is_auto_renew,payment_method_id,registered_via,avg_num_985,gender,total_secs,is_cancel,transaction_date
13.0667,1.48398e+09,0,30,1,0.933333,2014-07-14T00:00:00.000Z,++4RuqBw0Ss6bQU4oMxaRlbBPoWzoEiIZaxPM04Y4+U=,0.733333,149,149,5.73333,6.46667,nan,0,1,41,7,0.666667,,1978.66,0,1.4856e+09
61.1333,1.48755e+09,31,30,15,1.26667,2006-06-03T00:00:00.000Z,+/namlXq+u3izRjHCFJV4MgqcXcLidZYszVsROOq/y4=,1.4,149,149,29.1333,33.6,nan,0,1,34,9,1.26667,male,9395.27,0,1.48703e+09
19.8667,1.48764e+09,31,30,9,1.86667,2004-03-30T00:00:00.000Z,+0/X9tkmyHyet9X80G6GTrDFHnJqvai8d1ZPhayT0os=,1.06667,149,149,12.4667,67.8,nan,0,1,34,9,3.26667,male,17219,0,1.48703e+09
27.8,1.48751e+09,29,30,15,0.6,2008-03-22T00:00:00.000Z,+09YGn842g6h2EZUXe0VWeC4bBoCbDGfUboitc0vIHw=,0.333333,149,149,2.33333,33,nan,0,1,34,9,0.666667,male,8571.42,0,1.48703e+09
30.9333,1.48739e+09,24,30,5,4.4,2014-03-20T00:00:00.000Z,+0jTOa6KGPk1vtNTwRDMZc/McUo41AeuwV3ndo54Y+Q=,0.933333,149,149,10.7333,19.6,nan,0,1,23,9,0.533333,female,5353.78,0,1.48448e+09
46.6667,1.48743e+09,32,30,13,1.6,2015-03-16T00:00:00.000Z,+0l+FDuhyjaZnu0APnrg5L9QqgaRw4RmdQMvqOtKDmU=,1.66667,149,149,12.3333,40.6,nan,0,1,37,3,0.933333,male,10597.8,0,1.48656e+09
20.6667,1.48656e+09,0,30,5,0.4,2013-02-27T00:00:00.000Z,+0l/WkoOIugT69NYawwewSLZjIJ17kHIpDdWqcp53RI=,0.333333,149,149,1,21.5333,nan,0,1,40,3,0.133333,,5413.59,0,1.48647e+09
15,1.48764e+09,31,30,6,0.466667,2008-04-17T00:00:00.000Z,+2Df04hr61UUJijMM2xR97gtoQWWDJpnJVKQ7VMYN9o=,0.333333,180,180,1.26667,281.667,nan,0,1,36,9,1.86667,female,48663,0,1.4844e+09
18.6667,1.48603e+09,0,30,4,1.66667,2015-11-03T00:00:00.000Z,+2KZws+cYLzerLNA6dgCOpxKysRv4BQ8SiKtA0rV4QE=,0.866667,180,180,5.2,13.4667,nan,0,0.5,29,7,0.466667,,3910.75,0,1.485e+09
23.2667,1.48719e+09,0,30,1,2.73333,2012-12-17T00:00:00.000Z,+2eLsQv6T46iKwO+m+r6OFI2X3Oc9dGBMdti2COAe4w=,1.33333,99,99,8.93333,11.4,nan,0,1,41,7,3,,3921.46,0,1.48491e+09


In [57]:
spotify_test = spotify_test[:, ["msno","is_churn","city","bd","gender","registered_via","registered_init_time","date","avg_num_25","avg_num_50","avg_num_75","avg_num_985","avg_num_100","avg_num_unq","total_secs","payment_method_id","payment_plan_days","plan_list_price","actual_amount_paid","is_auto_renew","transaction_date","membership_expire_date","is_cancel"]]

In [58]:
spotify_test

msno,is_churn,city,bd,gender,registered_via,registered_init_time,date,avg_num_25,avg_num_50,avg_num_75,avg_num_985,avg_num_100,avg_num_unq,total_secs,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
++4RuqBw0Ss6bQU4oMxaRlbBPoWzoEiIZaxPM04Y4+U=,0,1,0,,7,2014-07-14T00:00:00.000Z,1.48398e+09,5.73333,0.933333,0.733333,0.666667,6.46667,13.0667,1978.66,41,30,149,149,1,1.4856e+09,nan,0
+/namlXq+u3izRjHCFJV4MgqcXcLidZYszVsROOq/y4=,0,15,31,male,9,2006-06-03T00:00:00.000Z,1.48755e+09,29.1333,1.26667,1.4,1.26667,33.6,61.1333,9395.27,34,30,149,149,1,1.48703e+09,nan,0
+0/X9tkmyHyet9X80G6GTrDFHnJqvai8d1ZPhayT0os=,0,9,31,male,9,2004-03-30T00:00:00.000Z,1.48764e+09,12.4667,1.86667,1.06667,3.26667,67.8,19.8667,17219,34,30,149,149,1,1.48703e+09,nan,0
+09YGn842g6h2EZUXe0VWeC4bBoCbDGfUboitc0vIHw=,0,15,29,male,9,2008-03-22T00:00:00.000Z,1.48751e+09,2.33333,0.6,0.333333,0.666667,33,27.8,8571.42,34,30,149,149,1,1.48703e+09,nan,0
+0jTOa6KGPk1vtNTwRDMZc/McUo41AeuwV3ndo54Y+Q=,0,5,24,female,9,2014-03-20T00:00:00.000Z,1.48739e+09,10.7333,4.4,0.933333,0.533333,19.6,30.9333,5353.78,23,30,149,149,1,1.48448e+09,nan,0
+0l+FDuhyjaZnu0APnrg5L9QqgaRw4RmdQMvqOtKDmU=,0,13,32,male,3,2015-03-16T00:00:00.000Z,1.48743e+09,12.3333,1.6,1.66667,0.933333,40.6,46.6667,10597.8,37,30,149,149,1,1.48656e+09,nan,0
+0l/WkoOIugT69NYawwewSLZjIJ17kHIpDdWqcp53RI=,0,5,0,,3,2013-02-27T00:00:00.000Z,1.48656e+09,1,0.4,0.333333,0.133333,21.5333,20.6667,5413.59,40,30,149,149,1,1.48647e+09,nan,0
+2Df04hr61UUJijMM2xR97gtoQWWDJpnJVKQ7VMYN9o=,0,6,31,female,9,2008-04-17T00:00:00.000Z,1.48764e+09,1.26667,0.466667,0.333333,1.86667,281.667,15,48663,36,30,180,180,1,1.4844e+09,nan,0
+2KZws+cYLzerLNA6dgCOpxKysRv4BQ8SiKtA0rV4QE=,0,4,0,,7,2015-11-03T00:00:00.000Z,1.48603e+09,5.2,1.66667,0.866667,0.466667,13.4667,18.6667,3910.75,29,30,180,180,0.5,1.485e+09,nan,0
+2eLsQv6T46iKwO+m+r6OFI2X3Oc9dGBMdti2COAe4w=,0,1,0,,7,2012-12-17T00:00:00.000Z,1.48719e+09,8.93333,2.73333,1.33333,3,11.4,23.2667,3921.46,41,30,99,99,1,1.48491e+09,nan,0


In [59]:
final_gbm_predictions = spotify_gbm.predict(spotify_test[1:])

gbm prediction progress: |████████████████████████████████████████████████| 100%


/home/ricardo/.local/lib/python3.5/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'registered_init_time' has levels not trained on: [2017-02-24T00:00:00.000Z]
  warnings.warn(w)
/home/ricardo/.local/lib/python3.5/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset is missing column 'num_25': substituting in a column of NaN
  warnings.warn(w)
/home/ricardo/.local/lib/python3.5/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset is missing column 'num_50': substituting in a column of NaN
  warnings.warn(w)
/home/ricardo/.local/lib/python3.5/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset is missing column 'num_75': substituting in a column of NaN
  warnings.warn(w)
/home/ricardo/.local/lib/python3.5/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset is missing column 'num_985': substituting in a column of NaN
  warnings.warn(w)
/home/ricardo/.local/lib/python3.5/site-packages/h2o/job.py:69: UserWarning

In [62]:
len(spotify_test[:,:])

196372

In [63]:
msnos_test = spotify_test[:,0]

In [64]:
msnos_test[10,0]

'+2oK/qWmYvAnfNZsVV5pdsJ9n6d/LZn6CdwiJajGZas='

In [65]:
msnos_test["prediction"] = final_gbm_predictions[2]

In [66]:
msnos_test

msno,prediction
++4RuqBw0Ss6bQU4oMxaRlbBPoWzoEiIZaxPM04Y4+U=,0.0029779
+/namlXq+u3izRjHCFJV4MgqcXcLidZYszVsROOq/y4=,0.00350683
+0/X9tkmyHyet9X80G6GTrDFHnJqvai8d1ZPhayT0os=,0.00350683
+09YGn842g6h2EZUXe0VWeC4bBoCbDGfUboitc0vIHw=,0.00350683
+0jTOa6KGPk1vtNTwRDMZc/McUo41AeuwV3ndo54Y+Q=,0.00388029
+0l+FDuhyjaZnu0APnrg5L9QqgaRw4RmdQMvqOtKDmU=,0.00347604
+0l/WkoOIugT69NYawwewSLZjIJ17kHIpDdWqcp53RI=,0.00416238
+2Df04hr61UUJijMM2xR97gtoQWWDJpnJVKQ7VMYN9o=,0.00612556
+2KZws+cYLzerLNA6dgCOpxKysRv4BQ8SiKtA0rV4QE=,0.0188944
+2eLsQv6T46iKwO+m+r6OFI2X3Oc9dGBMdti2COAe4w=,0.0027343


In [ ]:
spotify_test.nrows

In [ ]:
spotify_test["msno"].tolist()